In [110]:
# === Librerías para manejo de datos y visualización ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# === Scikit-learn para preprocesamiento y evaluación ===
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score

# === XGBoost para clasificación y regresión ===
from xgboost import XGBClassifier, XGBRegressor


In [111]:
import pandas as pd

# Carga individual
df_primera = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-primera-5-cambios.csv', sep=",")
df_premier = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-premier-5-cambios.csv', sep=",")
df_bundesliga = pd.read_csv('D:/TFG/codigo/datos/indicadores-filtrados-bundesliga-5-cambios.csv', sep=",")


# Unir en un solo DataFrame
df = pd.concat([df_primera, df_premier, df_bundesliga], ignore_index=True)

# === Eliminación de columnas irrelevantes o identificadores ===
df.drop(['jornada', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)
df.head()


,porcentaje local ganados en sitio,porcentaje local ganados en general,porcentaje local empatados en sitio,porcentaje local empatados en general,porcentaje local perdidos en sitio,porcentaje local perdidos en general,porcentaje visitante ganados en sitio,porcentaje visitante ganados en general,porcentaje visitante empatados en sitio,porcentaje visitante empatados en general,...,proporcion local cambios alineacion centrocampista en general,proporcion visitante cambios alineacion centrocampista en sitio,proporcion visitante cambios alineacion centrocampista en general,proporcion local cambios alineacion delantero sitio,proporcion local cambios alineacion delantero en general,proporcion visitante cambios alineacion delantero en sitio,proporcion visitante cambios alineacion delantero en general,resultado_local,resultado_visitante,resultado_partido
0,50.00,37.50,0.00,12.50,50.00,50.00,0.00,0.00,60.00,66.67,...,1.38,1.60,1.67,1.75,1.75,1.00,1.22,1,1,X
1,66.67,42.86,0.00,14.29,33.33,42.86,0.00,11.11,80.00,44.44,...,1.57,1.60,1.67,2.00,2.14,0.80,1.00,4,2,1
2,0.00,12.50,66.67,37.50,33.33,50.00,66.67,42.86,0.00,28.57,...,2.00,1.00,1.14,1.00,0.75,2.00,2.29,1,1,X
3,80.00,55.56,20.00,33.33,0.00,11.11,60.00,62.50,20.00,12.50,...,1.11,1.80,2.00,1.20,1.33,1.20,1.38,1,1,X
4,75.00,71.43,25.00,28.57,0.00,0.00,33.33,42.86,33.33,28.57,...,2.29,1.67,1.57,1.50,1.86,1.67,1.43,1,0,1


In [112]:
# === Separación de variables predictoras (X) ===
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

# === Variables objetivo ===
# Para clasificación multiclase: resultado del partido (victoria local, empate, victoria visitante)
y_resultado = df['resultado_partido'].replace({'1': 0, 'X': 1, '2': 2})  # Mapear etiquetas a valores numéricos

# Para regresión: goles del equipo local y visitante
y_goles_local = df['resultado_local']
y_goles_visitante = df['resultado_visitante']


In [113]:
# === Escalado de X con MinMaxScaler (opcional en XGBoost, pero útil para consistencia) ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === División de datos en entrenamiento y prueba (clasificación) ===
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_scaled, y_resultado, test_size=0.2, random_state=42
)

# === División de datos para predicción de goles del equipo local ===
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(
    X_scaled, y_goles_local, test_size=0.2, random_state=42
)

# === División de datos para predicción de goles del equipo visitante ===
X_train_vis, X_test_vis, y_train_vis, y_test_vis = train_test_split(
    X_scaled, y_goles_visitante, test_size=0.2, random_state=42
)


### PREDICCIONES RESULTADO

In [114]:
# === Entrenamiento del modelo de clasificación con XGBoost ===
clf_model = XGBClassifier(
    objective='multi:softmax',   # Predicción multiclase con etiquetas
    num_class=3,                 # 3 clases: victoria local, empate, victoria visitante
    eval_metric='mlogloss',      # Log-loss multiclase (funciona bien como métrica)
    use_label_encoder=False,
    random_state=42
)

# Ajustar el modelo
clf_model.fit(X_train_cls, y_train_cls)

# Predicción sobre el conjunto de prueba
y_pred_cls = clf_model.predict(X_test_cls)


c:\Users\Miguel\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:57:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [115]:
# === Precisión global ===
accuracy = accuracy_score(y_test_cls, y_pred_cls)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

# === Matriz de confusión ===
conf_matrix = confusion_matrix(y_test_cls, y_pred_cls)
print("Matriz de confusión:")
print(conf_matrix)

Precisión en el conjunto de prueba: 0.4322
Matriz de confusión:
[[109  40  21]
 [ 51  21  14]
 [ 50  25  23]]


In [116]:
# === Importancia de las variables del modelo de clasificación ===
importances = clf_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame ordenado por importancia
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 15 más importantes
print("Top 10 variables más importantes:")
print(feature_importance_df.head(10))


Top 10 variables más importantes:
                                              Variable  Importancia
1                  porcentaje local ganados en general     0.015342
129  proporcion local cambios centrocampistas a def...     0.011721
34            proporcion local goles marcados en sitio     0.011459
83            proporcion visitante posesion en general     0.010891
81                proporcion local posesion en general     0.009760
35          proporcion local goles marcados en general     0.009223
40        proporcion visitante goles marcados en sitio     0.009032
82              proporcion visitante posesion en sitio     0.008744
76                   proporcion local cambios en sitio     0.008534
41      proporcion visitante goles marcados en general     0.008358


In [117]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.4274


### GOLES EQUIPO LOCAL

In [118]:
# === Modelo de regresión para predecir los goles del equipo local ===
reg_local_model = XGBRegressor(
    objective='reg:squarederror',  # Error cuadrático medio, típico en regresión
    eval_metric='rmse',            # Métrica de evaluación: raíz del error cuadrático medio
    random_state=42
)

# Entrenar el modelo
reg_local_model.fit(X_train_loc, y_train_loc)

# Predicción sobre el conjunto de prueba
y_pred_loc = reg_local_model.predict(X_test_loc)

In [119]:
# === Evaluación del modelo de regresión para goles del equipo local ===
from sklearn.metrics import mean_squared_error, r2_score

# Error cuadrático medio
mse_loc = mean_squared_error(y_test_loc, y_pred_loc)
rmse_loc = np.sqrt(mse_loc)

# Coeficiente de determinación R²
r2_loc = r2_score(y_test_loc, y_pred_loc)

print(f"RMSE (Goles local): {rmse_loc:.4f}")
print(f"R² Score (Goles local): {r2_loc:.4f}")


RMSE (Goles local): 1.3787
R² Score (Goles local): -0.1828


In [120]:
# === Importancia de las variables en el modelo de regresión ===
importances = reg_local_model.feature_importances_
feature_names = X.columns

# Crear DataFrame ordenado por importancia
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 15 más importantes
print("Top 10 variables más importantes para la predicción de goles del equipo local:")
print(feature_importance_df.head(10))


Top 10 variables más importantes para la predicción de goles del equipo local:
                                             Variable  Importancia
35         proporcion local goles marcados en general     0.038802
84              proporcion local total tiros en sitio     0.017274
89        proporcion local corners a favor en general     0.016452
49       porcentaje local mas 2,5 marcados en general     0.015912
12                   proporcion local puntos en sitio     0.015504
13                 proporcion local puntos en general     0.014931
154  proporcion visitante cambios 76 a final en sitio     0.014477
80                 proporcion local posesion en sitio     0.014189
152         proporcion local cambios 76 a final sitio     0.013716
83           proporcion visitante posesion en general     0.012692


In [121]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.4205


### GOLES EQUIPO VISITANTE

In [122]:
# === Modelo de regresión para predecir los goles del equipo visitante ===
reg_visit_model = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    random_state=42
)

# Entrenar el modelo
reg_visit_model.fit(X_train_vis, y_train_vis)

# Predicción sobre el conjunto de prueba
y_pred_vis = reg_visit_model.predict(X_test_vis)


In [123]:
# Métricas de evaluación
rmse_vis = np.sqrt(mean_squared_error(y_test_vis, y_pred_vis))
r2_vis = r2_score(y_test_vis, y_pred_vis)

print(f"RMSE (goles visitante): {rmse_vis:.4f}")
print(f"R² Score (goles visitante): {r2_vis:.4f}")


RMSE (goles visitante): 1.1290
R² Score (goles visitante): -0.1198


In [124]:
# === Importancia de las variables en el modelo de regresión ===
importances = reg_visit_model.feature_importances_
feature_names = X.columns

# Crear DataFrame ordenado por importancia
feature_importance_df = pd.DataFrame({
    'Variable': feature_names,
    'Importancia': importances
}).sort_values(by='Importancia', ascending=False)

# Mostrar las 15 más importantes
print("Top 10 variables más importantes para la predicción de goles del equipo visitante:")
print(feature_importance_df.head(10))


Top 10 variables más importantes para la predicción de goles del equipo visitante:
                                              Variable  Importancia
41      proporcion visitante goles marcados en general     0.047056
163  proporcion visitante cambios alineacion centro...     0.022515
87         proporcion visitante total tiros en general     0.020481
100           proporcion local cambios amarillas sitio     0.017159
139  proporcion visitante cambios defensas a centro...     0.016607
143  proporcion visitante cambios antes descanso en...     0.016599
97      proporcion local cambios lesionados en general     0.014846
65   porcentaje visitante mas 1,5 encajados en general     0.014570
78               proporcion visitante cambios en sitio     0.014312
43     proporcion visitante goles encajados en general     0.013684


In [125]:
# === Suma de la importancia de variables relacionadas con la rotación ===
cambio_vars = feature_importance_df[feature_importance_df['Variable'].str.contains('cambio', case=False)]
suma_importancia_cambio = cambio_vars['Importancia'].sum()
print(f"Suma total de importancia de las variables relacionadas con la rotación: {suma_importancia_cambio:.4f}")

Suma total de importancia de las variables relacionadas con la rotación: 0.4962
